In [42]:
import pandas as pd
employees=pd.read_excel("20240108-employees-dec.xlsx", dtype={'Emp ID': "str"})


print(employees.columns)


employees["mail"]=employees["Email ID"]+"@infosys.com"
employees["mail"]=employees["mail"].str.lower()
# Construction de la colonne nom simple qui contient une concaténation du prénom et du nom en minuscule
employees["nomsimple"]= employees["Prénom"]+" "+employees["Nom"]
employees["nomsimple"]=employees["nomsimple"].str.lower().str.replace("é","e").replace("è","e").replace("ì","i")
employees["nomsimple"]=employees["nomsimple"].fillna("")

employees


Index(['Emp ID', 'Nom', 'Prénom', 'Email ID', 'email', 'location', 'Status'], dtype='object')


,Emp ID,Nom,Prénom,Email ID,email,location,Status,mail,nomsimple
0,4484,HEMMIGE,Raghu Ranganathan,RAGHURHEMMIGE,RAGHURHEMMIGE@infosys.com,Toulouse,Present in France,raghurhemmige@infosys.com,raghu ranganathan hemmige
1,15536,MENACHERRY,Antony,Antony_Menacherry,Antony_Menacherry@infosys.com,Marseille,Present in France,antony_menacherry@infosys.com,antony menacherry
2,15958,MOHANTY,Sachidananda,Sachidananda_Mohanty,Sachidananda_Mohanty@infosys.com,Marseille,Present in France,sachidananda_mohanty@infosys.com,sachidananda mohanty
3,17521,KAMATH,Ajit Venkatesh,Ajit_Kamath,Ajit_Kamath@infosys.com,Paris,Present in France,ajit_kamath@infosys.com,ajit venkatesh kamath
4,19406,DHARMAR,Shenbagarajan,Shenbagarajan_d,Shenbagarajan_d@infosys.com,Marseille,Present in France,shenbagarajan_d@infosys.com,shenbagarajan dharmar
...,...,...,...,...,...,...,...,...,...
504,12222,Rajesh Prabhakaran,Pillai,NaN,rajesh_pillai@infosys.com,Marseille,New to France,NaN,pillai rajes...
505,1314661,Pavan Shankar,Warpe,NaN,pavanshankar.warpe@infosys.com,Marseille,New to France,NaN,warpe pavan...
506,187930,Goutham,Muragavel,NaN,goutham_murugavel@infosys.com,Paris,New to France,NaN,muragavel gouth...
507,647645,Mushtaque Ali,Chenattu Kuzhiyan,NaN,mushtaque_c@infosys.com,Paris,New to France,NaN,chenattu kuzhiyan musht...


In [52]:
#Users est pris d'un extract Comitéo. : Liste des utilisateurs actifs et désactivés (sans adresse)
users=pd.read_csv("20240108_utilisateurs.csv",sep=";", dtype={'Matricule': "str"})

subset = ['Mail', 'Mail 2',"Matricule"]
users.loc[:, subset] = users.loc[:, subset].fillna("")
users=users[users["Etat du salarié"]=="Actif"]
users

users["mail"]=""

#Construction de la colonne mail qui sera utilisée pour le rapprochement. Elle contiendra le mail qui contient infosys.com
for i, row in users.iterrows():
    if "@infosys.com" in row["Mail"]:
        users.loc[i,"mail"]=row["Mail"]
    elif "@infosys.com" in  row["Mail 2"]:
        users.loc[i,"mail"]=row["Mail 2"]
# Minuscule
users["mail"]=users["mail"].str.lower()
# Construction de la colonne nom simple qui contient une concaténation du prénom et du nom en minuscule
users["nomsimple"]=users["Prénom"]+" "+users["Nom"]
users["nomsimple"]=users["nomsimple"].str.lower()
users
users[users.mail.str.contains("arnau")]
#employees[employees.nomsimple.str.contains("malivert")]["Emp ID"]

,User ID,Matricule,Identifiant,Civilité,Nom,Prénom,Etat du salarié,Mail,Mail 2,Site,...,Statut salarié,Date de naissance,IBAN,BIC,RUM,Date du RUM,Date de dernière connexion,Date de création,mail,nomsimple
163,3437323,1005910,afrancois,M.,Francois,Arnaud,Actif,afrancois@panaya.com,arnaud.francois@infosys.com,Paris,...,Cadolib_Noel_170 | Cadolib_Enfant_100 | Cadoli...,01/03/1979,NaN,NaN,NaN,NaN,15/12/2023,23/04/2019,arnaud.francois@infosys.com,arnaud francois
1036,6743089,530957,arnau.trn,M.,GARCIA CASES,Arnau,Actif,arnau.trn@infosys.com,,Toulouse,...,NA_70_Dec23,NaN,NaN,NaN,NaN,NaN,09/06/2023,30/05/2023,arnau.trn@infosys.com,arnau garcia cases


In [74]:
from IPython.core.display import HTML
trouves=[]
nontrouves=[]

#Premier rapprochement sur l'email
for i,ligne in employees.iterrows():   
    if  len(users.loc[users["mail"]==ligne.mail])>0:
        #print ("Trouvé ")
        indextrouve=users.loc[users["mail"]==ligne.mail].Identifiant.index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

#Second rapprochement sur le matricule.
nontrouvesrestant=[]
for i,ligne in employees.loc[nontrouves].iterrows():   
    #print(ligne["Emp No"])
    if  len(users.loc[users["Matricule"]==ligne["Emp ID"]])>0:
        #print ("Trouvé ")
        indextrouve=users.loc[users["Matricule"]==ligne["Emp ID"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouvesrestant.append(i)

nontrouves=[]
for i,ligne in employees.loc[nontrouvesrestant].iterrows():   
    #print(ligne["Emp No"])
    if  len(users.loc[users["nomsimple"]==ligne["nomsimple"]])>0:
        print ("Trouvé ")
        indextrouve=users.loc[users["nomsimple"]==ligne["nomsimple"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

    
employees.loc[nontrouves]  

users.loc[trouves,["Matricule","Identifiant"]].to_excel("users-chequecadeau.xlsx")

display(HTML("<h2>Employés non trouvés dans comitéo</h2>"))

employees.loc[nontrouves]  
acreer=pd.DataFrame(columns=["Identifiant\n(obligatoire - peut être identique à l’adresse email)"	,"Mail de communication"	,
                    "Prénom (obligatoire)",	"Nom (obligatoire)"	,"Actif (Actif/Inactif) (obligatoire)"	
                             ,"Civilité (Mme/M.) (obligatoire)",	
                    "Date de naissance (JJ/MM/AAAA)",	
                             "Mail secondaire",	"Autorisation des communications CSE (Oui/Non)"	,
                    "Autorisation des communications des Bons Plans (Oui/Non)"	,"Situation familiale (Célibataire, Marié(e), Pacsé(e), Divorcé(e), Veuf(ve), Concubinage)"	,
                    "Matricule"	,"Date d'entrée dans l'entreprise (JJ/MM/AAAA)"	,"Date de sortie de l'entreprise (JJ/MM/AAAA)"	,"Site de rattachement"	,
                    "Fonction dans l'entreprise",	                 "Service de l'entreprise",
                    "Type de contrat (Alternant, Apprenti, CDD, CDI, Externe, Retraité(e), Stagiaire)"	,"Téléphone principal (0101010101)"	,
                    "Téléphone secondaire (0101010101)"	,     "Commentaire"	,"Statut"	,"Quotient familial de l'année précédente",
                    "Quotient familial de l'année en cours"	,"Quotient familial de l'année suivante",	"Titre de l'adresse",	"Adresse de facturation par défaut (Oui/Non)",	
                    "Adresse de livraison par défaut (Oui/Non)",	"Adresse",	"Complément d'adresse"	,"Code Postal"	,"Ville",	
                    "Pays (Code ISO 2 lettres, ex: France -> FR)",	 "IBAN",	"BIC",	"Référence Unique de Mandat (RUM)",	"Date du mandat de prélèvement (JJ/MM/AAAA)"])
acreer["Identifiant\n(obligatoire - peut être identique à l’adresse email)"]=employees.loc[nontrouves]['email'].str.replace("@infosys.com","")
acreer["Mail de communication"]=employees.loc[nontrouves]['email']
acreer["Prénom (obligatoire)"]=employees.loc[nontrouves]['Prénom']
acreer["Nom (obligatoire)"]=employees.loc[nontrouves]['Nom']
acreer["Civilité (Mme/M.) (obligatoire)"]='M.'
acreer["Actif (Actif/Inactif) (obligatoire)"]='Actif'
acreer["Site de rattachement"]=employees.loc[nontrouves]['location']
employees.loc[nontrouves]
acreer
acreer.to_excel("acreer.xlsx")

In [59]:
from IPython.core.display import HTML
trouves=[]
nontrouves=[]

#Premier rapprochement sur l'email
for i,ligne in users.iterrows():   
    if  len(employees.loc[employees["mail"]==ligne.mail])>0:
        #print ("Trouvé ")
        indextrouve=employees.loc[employees["mail"]==ligne.mail]["Emp ID"].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

#Second rapprochement sur le matricule.
nontrouvesrestant=[]
for i,ligne in users.loc[nontrouves].iterrows():   
    if  len(employees.loc[employees["Emp ID"]==ligne["Matricule"]])>0:
        #print ("Trouvé ")
        indextrouve=employees.loc[employees["Emp ID"]==ligne["Matricule"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouvesrestant.append(i)

nontrouves=[]
for i,ligne in users.loc[nontrouvesrestant].iterrows():   
    #print(ligne["Emp No"])
    if  len(employees.loc[employees["nomsimple"]==ligne["nomsimple"]])>0:
        print ("Trouvé ")
        indextrouve=employees.loc[employees["nomsimple"]==ligne["nomsimple"]].index[0]
        #print( indextrouve)
        trouves.append(indextrouve)
    else:
        #print ("Non trouvé")
        nontrouves.append(i)

#employees.loc[nontrouves]  


display(HTML("<h2>Employés non trouvés dans employees a supprimer</h2>"))

users.loc[nontrouves]  

Trouvé 


,User ID,Matricule,Identifiant,Civilité,Nom,Prénom,Etat du salarié,Mail,Mail 2,Site,...,Statut salarié,Date de naissance,IBAN,BIC,RUM,Date du RUM,Date de dernière connexion,Date de création,mail,nomsimple
134,2354725,0000003,jmaraval4,Mme,Maraval,Jessica,Actif,maravaljessica@yahoo.fr,recepetion_toulouse@infosys.com,Toulouse,...,Cadolib_Noel_170 | Cadolib_NA_2021_100 | Cadol...,NaN,NaN,NaN,NaN,NaN,14/12/2023,11/09/2018,recepetion_toulouse@infosys.com,jessica maraval
428,3437950,,gpandian,M.,Pandian,Gopalakrishnan,Actif,kvpgops@hotmail.com,kvpgops@gmail.com,Paris,...,NaN,03/06/1991,NaN,NaN,NaN,NaN,26/12/2023,23/04/2019,,gopalakrishnan pandian
677,4445102,000002,mbenkaddour,Mme,Benkaddour,Mounia,Actif,mouniamls@gmail.com,reception.marseille@infosys.com,Marseille,...,Cadolib_Noel_170 | Cadolib_NA_2021_100 | Cadol...,NaN,NaN,NaN,NaN,NaN,14/12/2023,30/11/2020,reception.marseille@infosys.com,mounia benkaddour
700,4754796,604415,Vijayakumar_C02,M.,Vijayakumar,Chinnamuthu,Actif,Vijayakumar_C0221@infosys.com,vijayakumar_c0221@infosys.com,Paris,...,Cadolib_NA_2021_100 | Cadolib_Noel_Adulte_22,NaN,NaN,NaN,NaN,NaN,22/12/2023,14/06/2021,vijayakumar_c0221@infosys.com,chinnamuthu vijayakumar
785,5201126,000001,virginie_leroy,Mme,LEROY,Virginie,Actif,Virginie.angie.leroy@gmail.com,fr_reception@infosys.com,Paris,...,Cadolib_NA_2021_100 | Cadolib_Noel_Adulte_22,NaN,NaN,NaN,NaN,NaN,14/12/2023,14/12/2021,fr_reception@infosys.com,virginie leroy
824,5551731,635747,ThamimAnsari_M@infosys.com,M.,Musafar Gani,Thamim Ansari,Actif,ThamimAnsari_M2@infosys.com,,E001,...,Cadolib_Noel_Adulte_22,NaN,NaN,NaN,NaN,NaN,19/12/2023,25/04/2022,thamimansari_m2@infosys.com,thamim ansari musafar gani
871,5918510,699744,rbajpai,M.,Bajpai,Rajesh,Actif,no-reply@infosys.com,,Paris,...,Cadolib_Noel_Adulte_22,NaN,NaN,NaN,NaN,NaN,19/12/2023,20/09/2022,no-reply@infosys.com,rajesh bajpai
1029,6743082,530943,romeo.trn,M.,MALIVERT,Romeo,Actif,romeo01.trn@infosys.com,,Toulouse,...,NA_70_Dec23,NaN,NaN,NaN,NaN,NaN,13/06/2023,30/05/2023,romeo01.trn@infosys.com,romeo malivert
1036,6743089,530957,arnau.trn,M.,GARCIA CASES,Arnau,Actif,arnau.trn@infosys.com,,Toulouse,...,NA_70_Dec23,NaN,NaN,NaN,NaN,NaN,09/06/2023,30/05/2023,arnau.trn@infosys.com,arnau garcia cases
